In [1]:
import pandas as pd
import numpy as np
import time
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
file=pd.read_csv('title.rating.tsv', sep='\t', header=0)
data_rating = pd.DataFrame(file)
file=pd.read_csv('title.akas.tsv', sep='\t', header=0)
data_akas = pd.DataFrame(file)
file=pd.read_csv('title.basic.tsv', sep='\t', header=0)
data_basic = pd.DataFrame(file)

C:\Users\prath\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\prath\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_akas=data_akas[['titleId','language']]
lang=[]
for x in data_akas['language']:
    if x != r'\N':
        lang.append(x)
lang=list(set(lang))
lang.sort()

In [4]:
langs_used=[r'te' ,r'en', r'hi']
remove=[]
for x in range(0,len(data_akas['language'])):
        if data_akas['language'][x] not in langs_used:
            remove.append(x)
data_akas=data_akas.drop(remove)
data_akas=data_akas.drop_duplicates(['titleId'])

In [5]:
remove=[]
for w in range(0,len(data_basic)):
    if data_basic['titleType'][w]!=r'movie':
        remove.append(w)
data_basic=data_basic.drop(data_basic.index[remove])

In [6]:
data_basic=data_basic.drop(['endYear','runtimeMinutes','originalTitle'], axis = 1)

data_basic=data_basic.set_index(str('tconst'))
data_akas=data_akas.set_index(str('titleId'))

In [7]:
remove=[]
for t_ID in range(0,len(data_basic)):
    if data_basic['genres'][t_ID] == r'\N':# or data_basic['genres'][t_ID] == r'Documentary':
        remove.append(t_ID)

data_basic=data_basic.drop(data_basic.index[remove])

In [8]:
remove=[]
for t_ID in range(0,len(data_basic)):
    if data_basic['startYear'][t_ID] == r'\N' or int(data_basic['startYear'][t_ID]) < 1980:
        remove.append(t_ID)

data_basic=data_basic.drop(data_basic.index[remove])

In [9]:
selected_lang_movies=[]
remove=[]
title_id=list(data_akas.index.values)
tconst = list(data_basic.index.values)
for t_ID in range(0,len(data_basic)):
    if tconst[t_ID] not in title_id:
        remove.append(t_ID)

data_basic=data_basic.drop(data_basic.index[remove])

In [10]:
remove=[]
title_id=list(data_akas.index.values)
tconst = list(data_basic.index.values)
for t_ID in range(0,len(data_akas)):
    if title_id[t_ID] not in tconst:
        remove.append(t_ID)

data_akas=data_akas.drop(data_akas.index[remove])
data=data_akas.join(data_basic)

In [11]:
remove=[]
title_id=list(data.index.values)
t_id=list(data_rating.index.values)
for t_id in range(0,len(data_rating)):
    if data_rating['tconst'][t_id] not in title_id:
        remove.append(t_id)
        
data_rating=data_rating.drop(data_rating.index[remove])
data_rating=data_rating.set_index(str('tconst'))

In [12]:
remove=[]
title_id=list(data.index.values)
tconst = list(data_rating.index.values)
for t_ID in range(0,len(data)):
    if title_id[t_ID] not in tconst:
        remove.append(t_ID)

data=data.drop(data.index[remove])
t_id=['numVotes','averageRating']
data_rating=data_rating[t_id]
data=data.join(data_rating)

In [13]:
remove=[]
for t_ID in range(0,len(data)):
    if np.isnan(data['averageRating'][t_ID]):
        remove.append(t_ID)

data=data.drop(data.index[remove])

In [14]:
remove=[]
for t_ID in range(0,len(data)):
    if data['numVotes'][t_ID] <500:
        remove.append(t_ID)
        
data=data.drop(data.index[remove])

In [15]:
remove=[]
for t_ID in range(0,len(data)):
    if data['averageRating'][t_ID] <3:
        remove.append(t_ID)
#len(remove)
data=data.drop(data.index[remove])

In [16]:
remove=[]
for t_ID in range(0,len(data)):
    if data['isAdult'][t_ID] ==1:
        remove.append(t_ID)
data=data.drop(data.index[remove])
data=data.drop(['isAdult','language','titleType'], axis = 1)

In [17]:
genres=''
for t_id in range(len(data)):
    genres += data['genres'][t_id] + ','
genres = nltk.word_tokenize(genres)

movie_genre=[]
for i in genres:
    i = i.lower()
    movie_genre.append(i)
movie_genre = list(set(movie_genre))
if 'sci-fi' in movie_genre:
    movie_genre.remove('sci-fi')
    movie_genre.append('scifi')
movie_genre.remove(',')

In [18]:
for t_id in range(0,len(data)):
    genre = nltk.word_tokenize(data['genres'][t_id])
    genre = list(set(genre))
    if 'sci-fi' in genre:
        genre.remove('sci-fi')
        genre.append('scifi')
    if ',' in genre:
        genre.remove(',')
    genre_movie=''
    for word in genre:
        genre_movie += word + ' , '
    genre_movie=genre_movie[:-2]
    genre_movie = genre_movie.lower()  
    data['genres'][t_id] = genre_movie

C:\Users\prath\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
vector = CountVectorizer(vocabulary=movie_genre)
x = vector.fit_transform(list(data['genres'])).toarray()

genre_df = pd.DataFrame(x)
index = list(data.index.values)
genre_df.index=index
genre_df.columns=movie_genre
genre_df.index.name='titleId'

In [20]:
data.to_csv('Cleaned data\cleaned_data.csv')
genre_df.to_csv('Cleaned data\genre.csv')